In [1]:
%matplotlib inline
import os
import time
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from PIL import Image

import sys
sys.path.append('..')
import d2lzh_pytorch as d2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### 获取数据集
- ImageFolder读取路径中所有文件夹图像hotdog，not-hotdog,并赋予标签0，1

In [ ]:
data_dir = './Datasets/hotdog'
train_imgs = ImageFolder(os.path.join(data_dir, 'train'))
test_imgs = ImageFolder(os.path.join(data_dir, 'test'))

In [ ]:
hotdogs = [train_imgs[i][0] for i in range(8)]
not_hotdogs = [train_imgs[-i-1][0] for i in range(8)]
d2l.show_images(hotdogs + not_hotdogs, 2, 8, scale=1.4);

- 图像预处理（扩增&标准化）

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_augs = transforms.Compose([
    transforms.RandomResizedCrop(size=224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])
test_augs = transforms.Compose([
    transforms.Resize(size=256),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    normalize
])

#### 定义和初始化模型

In [3]:
# pretrained=True 自动下载并加载预训练模型的参数
pretrained_net = models.resnet18(pretrained=False)

In [4]:
print(pretrained_net.fc)

Linear(in_features=512, out_features=1000, bias=True)


- 将最后一层修改成需要的输出类别数
- 此时fc层将被随机初始化，其他层仍保存着与训练得到的参数

In [5]:
pretrained_net.fc = nn.Linear(512, 2)
print(pretrained_net.fc)

Linear(in_features=512, out_features=2, bias=True)


- 针对不同层设置不同的学习率
- 预训练的层用较小的学习率来微调
- 随机初始化的fc层用较大的学习率

In [6]:
output_params_id = list(map(id, pretrained_net.fc.parameters()))
feature_params = filter(lambda p: id(p) not in output_params_id, pretrained_net.parameters())

lr = 0.01
optimizer = optim.SGD(
    [{'params': feature_params},
    {'params': pretrained_net.fc.parameters(), 'lr': lr * 10}],
    lr=lr, weight_decay=0.001
)

- 微调模型

In [7]:
def train_fine_tuning(net, optimizer, batch_size=128, num_epochs=5):
    data_dir = './Datasets/hotdog'
    train_iter = DataLoader(ImageFolder(os.path.join(data_dir, 'train'),
                            transform=train_augs), batch_size, shuffle=True)
    test_iter = DataLoader(ImageFolder(os.path.join(data_dir, 'test'),
                          transform=test_augs), batch_size, shuffle=False)
    loss = nn.CrossEntropyLoss()
    d2l.train(train_iter, test_iter, net, loss, optimizer, device, num_epochs)

In [8]:
train_fine_tuning(pretrained_net, optimizer)

training on  cuda
Epoch 1, loss 3.1569, train acc 0.589, test acc 0.787, time 31.4 sec
Epoch 2, loss 0.2686, train acc 0.793, test acc 0.823, time 19.2 sec
Epoch 3, loss 0.1695, train acc 0.785, test acc 0.805, time 19.2 sec
Epoch 4, loss 0.0998, train acc 0.833, test acc 0.843, time 19.3 sec
Epoch 5, loss 0.0751, train acc 0.835, test acc 0.681, time 19.6 sec
